In [2]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from src.item_recommender import ItemRecommender

import re

## Part 1

Load the pickle file and set the index to the title, create a bag of words representataion and get predictions

In [3]:
df = pd.read_pickle('data/movie_data.pickle')
df.head()

,Title,Genre,Director,Actors,Key_words
0,The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]","[common, decency, finding, solace, number, two..."
1,The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]","[reluctant, son, aging, patriarch, clandestine..."
2,The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]","[grip, portrayed, career, tightens, son, vito,..."
3,The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]","[ability, dark, knight, must, accept, one, fig..."
4,12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]","[evidence, colleagues, forcing, reconsider, ju..."


In [4]:
df.set_index('Title', inplace = True)
df.head()

,Genre,Director,Actors,Key_words
Title,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton]","[common, decency, finding, solace, number, two..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan]","[reluctant, son, aging, patriarch, clandestine..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton]","[grip, portrayed, career, tightens, son, vito,..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart]","[ability, dark, knight, must, accept, one, fig..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb]","[evidence, colleagues, forcing, reconsider, ju..."


In [5]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        if col != 'Director':
            words += ' '.join(row[col])+ ' '
        else:
            words += row[col]+ ' '
    row['bag_of_words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

In [6]:
df.head()

,bag_of_words
Title,
The Shawshank Redemption,crime drama frankdarabont timrobbins morganfr...
The Godfather,crime drama francisfordcoppola marlonbrando a...
The Godfather: Part II,crime drama francisfordcoppola alpacino rober...
The Dark Knight,action crime drama christophernolan christia...
12 Angry Men,crime drama sidneylumet martinbalsam johnfied...


In [8]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)


#### Create a instance of my recommender class and get predictions

In [41]:
rec = ItemRecommender()
count_df = pd.DataFrame(count_matrix.todense(), index=indices.values)

In [42]:
rec.fit(count_df)

In [43]:
print(rec.get_recommendations('Fargo'))

['No Country for Old Men' 'The Departed' 'Rope' 'The Godfather'
 'Reservoir Dogs']


### Q 4)
Fill in code for the `create_user_profile` method.

In [46]:
profile = rec.get_user_profile(['The Godfather','The Godfather: Part II'])

In [58]:
profile

array([0., 0., 0., ..., 0., 0., 0.])

There is not much to see as this is a large matrix however I know one index 584 has values in it so we can look there.

In [59]:
profile[584]

4.0

In [66]:
count.get_feature_names()[584]

'crime'

It looks like the word crime shows up a lot in the 2 movies involved with this profile.   
    
    
### Q 5)

Fill in the `get_user_recommendation` and test it.

In [71]:
print(rec.get_user_recommendation(['The Godfather','The Godfather: Part II']))

['Goodfellas' 'Rope' 'Cool Hand Luke' 'Scarface' 'Fargo']


# Part 2)

In [72]:
news = pd.read_pickle('data/articles.pkl')

In [73]:
news.head()

,document_type,web_url,lead_paragraph,abstract,snippet,news_desk,word_count,source,section_name,subsection_name,_id,pub_date,print_page,headline,content
0,article,http://www.nytimes.com/2013/10/03/sports/footb...,You would think that in a symmetric zero-sum s...,None,You would think that in a symmetric zero-sum s...,Sports,347,The New York Times,Sports,Pro Football,524d4e3a38f0d8198974001f,2013-10-03T00:00:00Z,None,Week 5 Probabilities: Why Offense Is More Impo...,the original goal building model football fore...
1,article,http://www.nytimes.com/2013/10/03/us/new-immig...,House Democrats on Wednesday unveiled an immig...,House Democrats unveil immigration bill that p...,House Democrats on Wednesday unveiled an immig...,National,83,The New York Times,U.S.,None,524cf71338f0d8198973ff7b,2013-10-03T00:00:00Z,21,New Immigration Bill Put Forward,house unveiled immigration bill provides path ...
2,article,http://www.nytimes.com/2013/10/03/us/arizona-j...,A federal judge on Wednesday ordered the appoi...,Federal Judge Murray Snow orders the appointme...,A federal judge on Wednesday ordered the appoi...,National,160,The New York Times,U.S.,None,524cf50e38f0d8198973ff79,2013-10-03T00:00:00Z,21,Arizona: Judge Orders Monitor to Oversee Maric...,federal judge wednesday ordered appointment in...
3,article,http://www.nytimes.com/2013/10/03/us/texas-sta...,Texas has turned to a compounding pharmacy to ...,"Documents show that Texas, nation's most activ...",Texas has turned to a compounding pharmacy to ...,National,112,The New York Times,U.S.,None,524cf39a38f0d8198973ff78,2013-10-03T00:00:00Z,21,Texas: State Bought Execution Drugs From a Com...,texas nation’s active death-penalty state turn...
4,article,http://www.nytimes.com/2013/10/03/sports/tenni...,"Rafael Nadal, aiming to end Novak Djokovic’s r...",None,"Rafael Nadal, aiming to end Novak Djokovic’s r...",Sports,49,The New York Times,Sports,Tennis,524cf28b38f0d8198973ff73,2013-10-03T00:00:00Z,14,Nadal on Track for No. 1 Spot,rafael nadal aiming end novak djokovic’s run 1...


In [86]:
news.headline

0       Week 5 Probabilities: Why Offense Is More Impo...
1                        New Immigration Bill Put Forward
2       Arizona: Judge Orders Monitor to Oversee Maric...
3       Texas: State Bought Execution Drugs From a Com...
4                           Nadal on Track for No. 1 Spot
5                   Judge Halts Work on World Cup Stadium
6                 Bayern Munich Dominates Manchester City
7           American Leads in World Gymnastics All-Around
8                        Brodeur’s Starting Streak to End
9                              Vonn Is Close to Returning
10            Whitney Winner Out of Breeders’ Cup Classic
11      Eight Are Killed in Tennessee After Tire Blows...
12                Fuel From Landfill Methane Goes on Sale
13      On Fashion Runway, South Sudan Takes Steps Tow...
14             Preparing Teachers for the Urban Classroom
15                                  New York: Two Cities?
16                       A Blind Spot on Rearview Cameras
17            

In [74]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
vectorized_df = vectorizer.fit_transform(news.content)

In [79]:
news_recommender = ItemRecommender()
news_recommender.fit(vectorized_df, titles = news.headline)

In [87]:
print(news_recommender.get_recommendations('New York: Two Cities?'))

['A Historic Fair Housing Decision Reaffirmed' '3 Jobs and No Apartment?'
 'Two Approaches to Homelessness'
 'Have-Nots Squeezed and Stacked in Hong Kong' 'Who Wants to Buy a House?']


# Extra Credit 

In [83]:
news['content_and_title'] = news.content + news.headline
news.head(3)

,document_type,web_url,lead_paragraph,abstract,snippet,news_desk,word_count,source,section_name,subsection_name,_id,pub_date,print_page,headline,content,content_and_title
0,article,http://www.nytimes.com/2013/10/03/sports/footb...,You would think that in a symmetric zero-sum s...,None,You would think that in a symmetric zero-sum s...,Sports,347,The New York Times,Sports,Pro Football,524d4e3a38f0d8198974001f,2013-10-03T00:00:00Z,None,Week 5 Probabilities: Why Offense Is More Impo...,the original goal building model football fore...,the original goal building model football fore...
1,article,http://www.nytimes.com/2013/10/03/us/new-immig...,House Democrats on Wednesday unveiled an immig...,House Democrats unveil immigration bill that p...,House Democrats on Wednesday unveiled an immig...,National,83,The New York Times,U.S.,None,524cf71338f0d8198973ff7b,2013-10-03T00:00:00Z,21,New Immigration Bill Put Forward,house unveiled immigration bill provides path ...,house unveiled immigration bill provides path ...
2,article,http://www.nytimes.com/2013/10/03/us/arizona-j...,A federal judge on Wednesday ordered the appoi...,Federal Judge Murray Snow orders the appointme...,A federal judge on Wednesday ordered the appoi...,National,160,The New York Times,U.S.,None,524cf50e38f0d8198973ff79,2013-10-03T00:00:00Z,21,Arizona: Judge Orders Monitor to Oversee Maric...,federal judge wednesday ordered appointment in...,federal judge wednesday ordered appointment in...


In [84]:
vectorizer2 = TfidfVectorizer(stop_words='english', max_features=10000)
vectorizer2.fit(news.content_and_title)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=10000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [85]:
content_matrix = vectorizer2.transform(news.content)
headline_matrix = vectorizer2.transform(news.headline)

In [91]:
def weighted_vector(content_matrix, headline_matrix, alpha):
    return (content_matrix * alpha) + (headline_matrix * (1-alpha))

In [89]:
weighted_vector = weighted_vector(content_matrix, headline_matrix, .9)

In [90]:
weighted_recommender = ItemRecommender()
weighted_recommender.fit(weighted_vector, titles = news.headline)
print(weighted_recommender.get_recommendations('New York: Two Cities?'))

['New York Legislators’ Secret Income'
 'Minnesota: Orchestra Cancels New York Concerts'
 'YouTube Plans a Music Awards Show Live From New York'
 'Technology Firm From India Will Be New York Marathon Title Sponsor'
 'Movers and Shapers']
